# 1. Set up the notebook

Import all the modules we need.

In [1]:
# These are standard modules
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# This is a custom interface to the pybullet simulator
import ae483_drone

pybullet build time: Jul 21 2022 19:59:54


# 2. Start the simulator

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are three optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window - if you are recording videos or generating data, it is faster not to show the window;
* `width` and `height` (positive integers) are the dimensions of the simulation window - these will also be the dimensions of snapshots or videos that are generated.

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug. (If you would like to help eliminate this strange behavior - which is a consequence of issues with how pybullet interacts with MacOS - by contributing to the pybullet open-source project, contact [Prof. Bretl](mailto:tbretl@illinois.edu).)

In [5]:
simulator = ae483_drone.Simulator(
    display=True,
    width=640,
    height=480,
)

## 3.1 Flight test (template)

Each time you duplicate this section, replace the title with a brief description of your current flight test.

### 3.1.1 Create a client

The "client" specifies the desired position and yaw angle at a given time.

In [3]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        o_x = 0.
        o_y = 0.
        o_z = 0.3
        psi = 0.
        
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

### 3.1.2 Create a controller

The "controller" specifies the motor power commands at a given state and setpoint.

In [4]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        
        tau_x = 1.00000000 * (o_y - o_y_des) -5.42958931 * phi + 1.45153364 * v_y -1.00005429 * w_x
        tau_y = -1.00000000 * (o_x - o_x_des) -5.42958931 * theta -1.45153364 * v_x -1.00005429 * w_y
        tau_z = -1.00000000 * psi -1.00002000 * w_z
        f_z = -1.00000000 * (o_z - o_z_des) -1.03150376 * v_z
        
        m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        
        return m_1, m_2, m_3, m_4

### 3.1.3 Add a drone with this client and controller to the simulator

Remove all existing drones from the simulator.

In [ ]:
simulator.clear_drones()

Add a new drone to the simulator.

There are three required arguments:

* the name of the drone (a string), for example `my_drone`
* the class that defines the client, for example `RobotClient`
* the class that defines the controller, for example `RobotController`

There is one optional argument that allows you to change the appearance of the drone (if you want) - this is useful if you simulate more than one drone at a time:

* `rgba` is a list of four numbers between 0 and 1 (red, green, blue, alpha) that define an [RGBA color](https://en.wikipedia.org/wiki/RGBA_color_model)

There are eight optional arguments that allow you to change the physical parameters that govern the drone (mass, moments of inertia, and so forth). You **must** change the values of these arguments if you want the simulated drone to match your real drone.

* `m`, `J_x`, `J_y`, `J_z`, `g`, `l`, `k_F`, `k_M` are scalar parameters

In [8]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m = 0.0312,
    J_x = 1.57e-05,
    J_y = 1.60e-05,
    J_z = 2.05e-05,
    g=9.81,
    k_F = 2.04e-06,
    k_M = 6.33e-09,
    l = .0325 
)

Set the initial state of this drone. Here, as an example, we start the drone near the desired position that was specified by the client.

In [9]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.01,
        'o_y': -0.02,
        'o_z': 0.27,
        'psi': 0.,
        'theta': 0.,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
    },
)

You could repeat this process to add more drones, if you want to test more than one client and controller (or more than one initial state) at a time.

### 3.1.4 Set the camera view (optional)

Here is how to make the camera always look at a certain point (in this case, a point that is 0.3 meters above the origin of the world frame):

In [10]:
simulator.set_camera_target([0.0, 0.0, 0.3])

Here is how to make the camera always look at a certain drone (in this case, the one called `my_drone`):

In [11]:
simulator.set_camera_target('my_drone')

Here is how to get a top view:

In [12]:
simulator.camera_topview()

Here is how to get a side view:

In [ ]:
simulator.camera_sideview()

Here is how to change the yaw angle of the camera (i.e., to make the camera rotate about the point it is looking at):

In [ ]:
simulator.set_camera_yaw(45)

Here is how to change the distance between the camera and the point it is looking at:

In [ ]:
simulator.set_camera_distance(1.0)

### 3.1.5 Run the simulator

Here is how to run the simulator for 10 seconds while saving data to the file `simulation_1_data.json` and saving video to the file `simulation_1_video.json`.

If you do not need to save video and want this to run much faster, then specify `video_filename=None`.

In [14]:
simulator.run(
    max_time=10.,
    data_filename='simulation_1_data.json',
    video_filename='simulation_1_video.mov',
)

Creating a video with name simulation_1_video.mov and fps 100
 100 / 1000
 200 / 1000
 300 / 1000
 400 / 1000
 500 / 1000
 600 / 1000
 700 / 1000
 800 / 1000
 900 / 1000
 1000 / 1000
Completed 1000 time steps in 77.5421 seconds (12.8962 time steps per second)


Be careful! Both the data file and the video file will be overwritten if they already exist. We suggest you use a different name for each experiment.

## 3.2 Flight test

Each time you duplicate this section, replace the title with a brief description of your current flight test.

### 3.2.1 Create a client

The "client" specifies the desired position and yaw angle at a given time.

In [8]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        o_x = 0.
        o_y = 0.
        o_z = 0.3
        psi = 0.
        
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

### 3.2.2 Create a controller

The "controller" specifies the motor power commands at a given state and setpoint.

In [43]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        
        tau_x = 0.00066300 * (o_y - o_y_des) -0.00439973 * phi + 0.00101699 * v_y -0.00076008 * w_x
        tau_y = -0.00066300 * (o_x - o_x_des) -0.00440388 * theta -0.00101727 * v_x -0.00076190 * w_y
        tau_z = -0.00000206 * psi -0.00000941 * w_z
        f_z = -0.00937624 * (o_z - o_z_des) -0.02436942 * v_z + 0.30607200
        
        m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        
        return m_1, m_2, m_3, m_4

### 3.2.3 Add a drone with this client and controller to the simulator

Remove all existing drones from the simulator.

In [44]:
simulator.clear_drones()

Add a new drone to the simulator.

There are three required arguments:

* the name of the drone (a string), for example `my_drone`
* the class that defines the client, for example `RobotClient`
* the class that defines the controller, for example `RobotController`

There is one optional argument that allows you to change the appearance of the drone (if you want) - this is useful if you simulate more than one drone at a time:

* `rgba` is a list of four numbers between 0 and 1 (red, green, blue, alpha) that define an [RGBA color](https://en.wikipedia.org/wiki/RGBA_color_model)

There are eight optional arguments that allow you to change the physical parameters that govern the drone (mass, moments of inertia, and so forth). You **must** change the values of these arguments if you want the simulated drone to match your real drone.

* `m`, `J_x`, `J_y`, `J_z`, `g`, `l`, `k_F`, `k_M` are scalar parameters

In [45]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m = 0.0312,
    J_x = 1.57e-05,
    J_y = 1.60e-05,
    J_z = 2.05e-05,
    g=9.81,
    k_F = 2.04e-06,
    k_M = 6.33e-09,
    l = .0325 
)

Set the initial state of this drone. Here, as an example, we start the drone near the desired position that was specified by the client.

In [46]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.01,
        'o_y': -0.02,
        'o_z': 0.27,
        'psi': 0.,
        'theta': 0.,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
    },
)

You could repeat this process to add more drones, if you want to test more than one client and controller (or more than one initial state) at a time.

### 3.2.4 Set the camera view (optional)

Here is how to make the camera always look at a certain point (in this case, a point that is 0.3 meters above the origin of the world frame):

In [28]:
simulator.set_camera_target([0.0, 0.0, 0.3])

Here is how to make the camera always look at a certain drone (in this case, the one called `my_drone`):

In [29]:
simulator.set_camera_target('my_drone')

Here is how to get a top view:

In [21]:
simulator.camera_topview()

Here is how to get a side view:

In [30]:
simulator.camera_sideview()

Here is how to change the yaw angle of the camera (i.e., to make the camera rotate about the point it is looking at):

In [ ]:
simulator.set_camera_yaw(45)

Here is how to change the distance between the camera and the point it is looking at:

In [ ]:
simulator.set_camera_distance(1.0)

### 3.2.5 Run the simulator

Here is how to run the simulator for 10 seconds while saving data to the file `simulation_x_data.json` and saving video to the file `simulation_x_video.json`.

If you do not need to save video and want this to run much faster, then specify `video_filename=None`.

In [47]:
simulator.run(
    max_time=10.,
    data_filename='simulation_2_data.json',
    video_filename='simulation_2_video.mov',
)

Creating a video with name simulation_2_video.mov and fps 100
 100 / 1000
 200 / 1000
 300 / 1000
 400 / 1000
 500 / 1000
 600 / 1000
 700 / 1000
 800 / 1000
 900 / 1000
 1000 / 1000
Completed 1000 time steps in 56.1586 seconds (17.8067 time steps per second)


Be careful! Both the data file and the video file will be overwritten if they already exist. We suggest you use a different name for each experiment.

## 3.3 Flight test

Each time you duplicate this section, replace the title with a brief description of your current flight test.

### 3.3.1 Create a client

The "client" specifies the desired position and yaw angle at a given time.

In [6]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        o_x = 0.
        o_y = 0.
        o_z = 0.3
        psi = 0.
        
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

### 3.3.2 Create a controller

The "controller" specifies the motor power commands at a given state and setpoint.

In [7]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        
        tau_x = 0.00066300 * (o_y - o_y_des) -0.00439973 * phi + 0.00101699 * v_y -0.00076008 * w_x
        tau_y = -0.00066300 * (o_x - o_x_des) -0.00440388 * theta -0.00101727 * v_x -0.00076190 * w_y
        tau_z = -0.00002057 * psi -0.00002912 * w_z
        f_z = -0.29650261 * (o_z - o_z_des) -0.16520637 * v_z + 0.30607200

        m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        
        return m_1, m_2, m_3, m_4

### 3.3.3 Add a drone with this client and controller to the simulator

Remove all existing drones from the simulator.

In [8]:
simulator.clear_drones()

Add a new drone to the simulator.

There are three required arguments:

* the name of the drone (a string), for example `my_drone`
* the class that defines the client, for example `RobotClient`
* the class that defines the controller, for example `RobotController`

There is one optional argument that allows you to change the appearance of the drone (if you want) - this is useful if you simulate more than one drone at a time:

* `rgba` is a list of four numbers between 0 and 1 (red, green, blue, alpha) that define an [RGBA color](https://en.wikipedia.org/wiki/RGBA_color_model)

There are eight optional arguments that allow you to change the physical parameters that govern the drone (mass, moments of inertia, and so forth). You **must** change the values of these arguments if you want the simulated drone to match your real drone.

* `m`, `J_x`, `J_y`, `J_z`, `g`, `l`, `k_F`, `k_M` are scalar parameters

In [9]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m = 0.0312,
    J_x = 1.57e-05,
    J_y = 1.60e-05,
    J_z = 2.05e-05,
    g=9.81,
    k_F = 2.04e-06,
    k_M = 6.33e-09,
    l = .0325 
)

Set the initial state of this drone. Here, as an example, we start the drone near the desired position that was specified by the client.

In [10]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.01,
        'o_y': -0.02,
        'o_z': 0.27,
        'psi': 0.,
        'theta': 0.,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
    },
)

You could repeat this process to add more drones, if you want to test more than one client and controller (or more than one initial state) at a time.

### 3.3.4 Set the camera view (optional)

Here is how to make the camera always look at a certain point (in this case, a point that is 0.3 meters above the origin of the world frame):

In [28]:
simulator.set_camera_target([0.0, 0.0, 0.3])

Here is how to make the camera always look at a certain drone (in this case, the one called `my_drone`):

In [29]:
simulator.set_camera_target('my_drone')

Here is how to get a top view:

In [21]:
simulator.camera_topview()

Here is how to get a side view:

In [30]:
simulator.camera_sideview()

Here is how to change the yaw angle of the camera (i.e., to make the camera rotate about the point it is looking at):

In [ ]:
simulator.set_camera_yaw(45)

Here is how to change the distance between the camera and the point it is looking at:

In [ ]:
simulator.set_camera_distance(1.0)

### 3.3.5 Run the simulator

Here is how to run the simulator for 10 seconds while saving data to the file `simulation_x_data.json` and saving video to the file `simulation_x_video.json`.

If you do not need to save video and want this to run much faster, then specify `video_filename=None`.

In [11]:
simulator.run(
    max_time=10.,
    data_filename='simulation_3_data.json',
    video_filename='simulation_3_video.mov',
)

Creating a video with name simulation_3_video.mov and fps 100
 100 / 1000
 200 / 1000
 300 / 1000
 400 / 1000
 500 / 1000
 600 / 1000
 700 / 1000
 800 / 1000
 900 / 1000
 1000 / 1000
Completed 1000 time steps in 54.2016 seconds (18.4496 time steps per second)


Be careful! Both the data file and the video file will be overwritten if they already exist. We suggest you use a different name for each experiment.

# 3. Run experiments

## 3.x Flight test (template)

Each time you duplicate this section, replace the title with a brief description of your current flight test.

### 3.x.1 Create a client

The "client" specifies the desired position and yaw angle at a given time.

In [ ]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        o_x = 0.
        o_y = 0.
        o_z = 0.3
        psi = 0.
        
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

### 3.x.2 Create a controller

The "controller" specifies the motor power commands at a given state and setpoint.

In [ ]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        
        # FIXME: Add code here to compute net torques and net force
        #
        #  tau_x = ...
        #  tau_y = ...
        #  tau_z = ...
        #  f_z = ...
        
        # FIXME: Replace code here to compute motor power commands
        m_1 = 0.
        m_2 = 0.
        m_3 = 0.
        m_4 = 0.
        
        return m_1, m_2, m_3, m_4

### 3.x.3 Add a drone with this client and controller to the simulator

Remove all existing drones from the simulator.

In [ ]:
simulator.clear_drones()

Add a new drone to the simulator.

There are three required arguments:

* the name of the drone (a string), for example `my_drone`
* the class that defines the client, for example `RobotClient`
* the class that defines the controller, for example `RobotController`

There is one optional argument that allows you to change the appearance of the drone (if you want) - this is useful if you simulate more than one drone at a time:

* `rgba` is a list of four numbers between 0 and 1 (red, green, blue, alpha) that define an [RGBA color](https://en.wikipedia.org/wiki/RGBA_color_model)

There are eight optional arguments that allow you to change the physical parameters that govern the drone (mass, moments of inertia, and so forth). You **must** change the values of these arguments if you want the simulated drone to match your real drone.

* `m`, `J_x`, `J_y`, `J_z`, `g`, `l`, `k_F`, `k_M` are scalar parameters

In [ ]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m=0.032,   # <-- FIXME
    J_x=1e-5,  # <-- FIXME
    J_y=1e-5,  # <-- FIXME
    J_z=2e-5,  # <-- FIXME
    g=9.81,
    l=0.035,   # <-- FIXME
    k_F=2e-6,  # <-- FIXME
    k_M=1e-8,  # <-- FIXME
)

Set the initial state of this drone. Here, as an example, we start the drone near the desired position that was specified by the client.

In [ ]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.01,
        'o_y': -0.02,
        'o_z': 0.27,
        'psi': 0.,
        'theta': 0.,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
    },
)

You could repeat this process to add more drones, if you want to test more than one client and controller (or more than one initial state) at a time.

### 3.x.4 Set the camera view (optional)

Here is how to make the camera always look at a certain point (in this case, a point that is 0.3 meters above the origin of the world frame):

In [ ]:
simulator.set_camera_target([0.0, 0.0, 0.3])

Here is how to make the camera always look at a certain drone (in this case, the one called `my_drone`):

In [ ]:
simulator.set_camera_target('my_drone')

Here is how to get a top view:

In [ ]:
simulator.camera_topview()

Here is how to get a side view:

In [ ]:
simulator.camera_sideview()

Here is how to change the yaw angle of the camera (i.e., to make the camera rotate about the point it is looking at):

In [ ]:
simulator.set_camera_yaw(45)

Here is how to change the distance between the camera and the point it is looking at:

In [ ]:
simulator.set_camera_distance(1.0)

### 3.x.5 Run the simulator

Here is how to run the simulator for 10 seconds while saving data to the file `simulation_x_data.json` and saving video to the file `simulation_x_video.json`.

If you do not need to save video and want this to run much faster, then specify `video_filename=None`.

In [ ]:
simulator.run(
    max_time=10.,
    data_filename='simulation_x_data.json',
    video_filename='simulation_x_video.mov',
)

Be careful! Both the data file and the video file will be overwritten if they already exist. We suggest you use a different name for each experiment.